In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("/home/jiaxingliu/workspace/swe-pdb/rllm/data/rl_1125.parquets")

In [6]:
df.iloc[0].to_json('test.json')

In [26]:
df.iloc[0].get('extra_info')['files'][0]

{'content': "from solution import *\n\nimport math\n\nfrom solution import *\n\nimport math\n\nfrom solution import *\n\nimport math\n\nfrom solution import display_board, is_winner, is_draw, make_move\n\ndef test_display_board():\n    board = [['-' for _ in range(5)] for _ in range(5)]\n    # This is a visual test, make sure the output looks like a 5x5 grid with '-'\n    display_board(board)\n\ndef test_is_winner_horizontal():\n    board = [\n        ['X', 'X', 'X', 'X', 'X'],\n        ['-', '-', '-', '-', '-'],\n        ['-', '-', '-', '-', '-'],\n        ['-', '-', '-', '-', '-'],\n        ['-', '-', '-', '-', '-'],\n    ]\n    assert is_winner(board, 'X')\n\ndef test_is_winner_vertical():\n    board = [\n        ['O', '-', '-', '-', '-'],\n        ['O', '-', '-', '-', '-'],\n        ['O', '-', '-', '-', '-'],\n        ['O', '-', '-', '-', '-'],\n        ['O', '-', '-', '-', '-'],\n    ]\n    assert is_winner(board, 'O')\n\ndef test_is_winner_diagonal():\n    board = [\n        ['X'

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("/data/jiaxingliu/checkpoints/qwen3-14b/step300/huggingface")
tokenizer.apply_chat_template(df.iloc[0]["prompt"], add_generation_prompt=True, tokenize=False)

'<|im_start|>assistant\n'

In [8]:
print(tokenizer.chat_template)

{%- if tools %}
    {{- '<|im_start|>system\n' }}
    {%- if messages[0].role == 'system' %}
        {{- messages[0].content + '\n\n' }}
    {%- endif %}
    {{- "# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>" }}
    {%- for tool in tools %}
        {{- "\n" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- "\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call><|im_end|>\n" }}
{%- else %}
    {%- if messages[0].role == 'system' %}
        {{- '<|im_start|>system\n' + messages[0].content + '<|im_end|>\n' }}
    {%- endif %}
{%- endif %}
{%- set ns = namespace(multi_step_tool=true, last_query_index=messages|length - 1) %}
{%- for message in messages[::-1] %}
    {%- set index = (messages|length - 

In [9]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "run_tests",
            "description": "运行项目测试套件并返回失败详情",
            "parameters": {
                "type": "object",
                "properties": {
                    "command": {
                        "type": "string",
                        "description": "完整测试命令，例如 `pytest tests/`"
                    }
                },
                "required": ["command"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "git_apply_patch",
            "description": "将补丁内容应用到仓库",
            "parameters": {
                "type": "object",
                "properties": {
                    "patch": {
                        "type": "string",
                        "description": "标准 unified diff 文本"
                    }
                },
                "required": ["patch"]
            }
        }
    }
]

messages = [
    {"role": "system", "content": "你是经验丰富的软件调试代理，请按步骤修复问题。"},
    {
        "role": "user",
        "content": "项目里的 `tests/test_math.py` 有用例挂了，先把失败原因找出来。"
    },
    {
        "role": "assistant",
        "content": '<tool_call name="run_tests">{"command": "pytest tests/test_math.py"}</tool_call>'
    },
    {
        "role": "tool",
        "name": "run_tests",
        "content": "FAIL test_division_zero ... ZeroDivisionError: division by zero"
    },
    {
        "role": "assistant",
        "content": "测试在除零逻辑上崩了，我需要改 `math_utils.py`，请给我补丁。"
    },
    {
        "role": "user",
        "content": "好的，修复完要重新跑一次测试确认。"
    }
]

In [12]:
print(tokenizer.apply_chat_template(messages,tools,add_generation_prompt=True, tokenize=False))

<|im_start|>system
你是经验丰富的软件调试代理，请按步骤修复问题。

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "run_tests", "description": "运行项目测试套件并返回失败详情", "parameters": {"type": "object", "properties": {"command": {"type": "string", "description": "完整测试命令，例如 `pytest tests/`"}}, "required": ["command"]}}}
{"type": "function", "function": {"name": "git_apply_patch", "description": "将补丁内容应用到仓库", "parameters": {"type": "object", "properties": {"patch": {"type": "string", "description": "标准 unified diff 文本"}}, "required": ["patch"]}}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}
</tool_call><|im_end|>
<|im_start|>user
项目里的 `tests/test_math.py` 有用例挂了，先把失败原因找出来。<|im_end|>
<|im_start|>assistant
<tool_call n